**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [21]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import tensorflow as tf
from collections import deque
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape, Flatten,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [22]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [23]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
    
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon: # np.random.rand() returns a single random float
                a = np.random.randint(0, self.n_action, size=1)[0] # One number between 0 and n_act
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act gives the action that the agent has to do. Epsilon is essential in this function and we will explain why. 

In the situation where there is an agent in an environment that wants to optimize the reward, there are two kind of actions: Exploitation and Exploration. 
Exploration is all about finding more information about an environment, whereas exploitation is exploiting already known information to maximize the rewards.

If at the beginning, we don't "force" the algorithm to to explorations actions, it will not find the optimal actions to maximize rewards. So, we need an $ \epsilon $ term which will manage the tradeoff between exploration and exploitation. 

At each step, we generate a random number r. If $ r > \epsilon $, then we will do “exploitation” (this means we use what we already know to select the best action at each step). Else, we’ll do exploration.

The idea is that we must have a big epsilon at the beginning of the training of our model. Then, reduce it progressively as the agent becomes more confident at estimating values.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [24]:
#test = np.concatenate((np.zeros((10,10,1)), np.zeros((10, 10,1))), axis=2)
#print(np.reshape(test,[2,5,5,4]))


In [25]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the mouse
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128 # 
        b[self.board>0,0] = 256  # For indices where self.board > 0, = 256 for b
        b[self.board < 0, 2] = 256 # Same idea
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state # Dim: 5 x 5 x 10

The following elements are important because they correspond to the hyper parameters for this project:

In [26]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The goal of the array position is to have the information: where is the rat on the board at a given time. For each call of the function "act", the array is updated by setting to zeros all the elements except the position of the rat that is set to 1. The goal of the array is to keep track of the position of the mouse.

The array board defines the board on which the rat moves around, taking into account all the rewards (0, 0.5 or -1) depending on if the mouse in on a poisonous cell, a neutral one or a cheese. 

The rat has a partial view of the board (visibility of 2 cells from its current position: self.x and self.y)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [27]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [28]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))


In [29]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 2.5/2.0. Average score (0.5)
Win/lose count 2.5/3.0. Average score (0.0)
Win/lose count 6.5/13.0. Average score (-2.1666666666666665)
Win/lose count 5.0/5.0. Average score (-1.625)
Win/lose count 5.5/7.0. Average score (-1.6)
Win/lose count 2.5/4.0. Average score (-1.5833333333333333)
Win/lose count 4.0/10.0. Average score (-2.2142857142857144)
Win/lose count 6.0/7.0. Average score (-2.0625)
Win/lose count 6.5/9.0. Average score (-2.111111111111111)
Win/lose count 5.5/10.0. Average score (-2.35)
Win/lose count 3.0/6.0. Average score (-2.409090909090909)
Win/lose count 7.5/8.0. Average score (-2.25)
Win/lose count 4.5/7.0. Average score (-2.269230769230769)
Win/lose count 4.0/2.0. Average score (-1.9642857142857142)
Win/lose count 4.0/8.0. Average score (-2.1)
Win/lose count 5.0/8.0. Average score (-2.15625)
Win/lose count 7.0/10.0. Average score (-2.2058823529411766)
Win/lose count 8.0/5.0. Average score (-1.9166666666666667)
Win/lose count 3.5/7.0. Average score (-2.0)


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




The action-value function can be written as:

$ Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> \\
 = E_{p^{\pi}}[r(s,a) + \gamma Q^{\pi}(s',a')]   $
 
 Then, $ Q^*(s,a) = max_{\pi}~E_{p^{\pi}}[r(s,a) + \gamma Q^{\pi}(s',a')] \\
         = E_{\pi *}[r(s,a) + \gamma  Q^{\pi ^*}(s',a')] \\
         = E_{p^{\pi}}[r(s,a) + \gamma max_{a'} Q^{*}(s',a')]
 $
 
 


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [30]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list() 
    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def random_access(self):
        idx = np.random.randint(0,len(self.memory))
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [31]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [32]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
    

    def learned_act(self, s):
        
        s = np.reshape(s,[1,5,5,self.n_state])
        # Predict the reward value based on the given state
        act_values = self.model.predict(s)
        return np.argmax(act_values[0])

    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        s_ = np.reshape(s_,[1,5,5,self.n_state])
        n_s_ = np.reshape(n_s_,[1,5,5,self.n_state])
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            target = r_
            # Take one of the elements of self.memory
            minibatch = self.memory.random_access()
            s_= minibatch[0]
            n_s_= minibatch[1]
            a_ = minibatch[2]
            r_= minibatch[3]
            game_over_= minibatch[4]
            
            input_states[i]= s_
            
            if game_over_:
                target= r_

            else:               
                target = (r_ + self.discount * np.amax(self.model.predict(n_s_)[0]))
            
            target_f = self.model.predict(s_) 
            
            # For each move, the estimated reward (list of list of 4 elements)
            # target_f[0] is the same but with just a list
            # target_f[0][a_] takes the position in the list (0,1,2 or 3) of a_ and gives the 
            # value of target
            target_f[0][a_] = target
            self.model.fit(s_, target_f, epochs=1, verbose=0)
            
            target_q[i]=target_f
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            

class DQN_FC(DQN):
    def __init__(self, lr=0.1,  *args, **kwargs):
        super(DQN_FC, self).__init__(*args,**kwargs)
        
        # NN Model
        state_size = 50
        
        model = Sequential()
        model.add(Reshape((state_size,), input_shape=(5,5,self.n_state)))
        model.add(Dense(512, input_dim=state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=lr))
        self.model = model
        

In [33]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=0.1, grid_size=10, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/020 | Loss 1.1151 | Win/lose count 3.0/1.0 (2.0)
Epoch 001/020 | Loss 0.0638 | Win/lose count 5.0/10.0 (-5.0)
Epoch 002/020 | Loss 0.1453 | Win/lose count 8.0/12.0 (-4.0)
Epoch 003/020 | Loss 0.1621 | Win/lose count 7.0/12.0 (-5.0)
Epoch 004/020 | Loss 0.1266 | Win/lose count 10.0/12.0 (-2.0)
Epoch 005/020 | Loss 0.1132 | Win/lose count 4.5/19.0 (-14.5)
Epoch 006/020 | Loss 0.1612 | Win/lose count 13.0/17.0 (-4.0)
Epoch 007/020 | Loss 1.0815 | Win/lose count 8.0/16.0 (-8.0)
Epoch 008/020 | Loss 0.0412 | Win/lose count 10.0/9.0 (1.0)
Epoch 009/020 | Loss 0.2122 | Win/lose count 11.0/18.0 (-7.0)
Epoch 010/020 | Loss 0.0758 | Win/lose count 7.5/5.0 (2.5)
Epoch 011/020 | Loss 0.2739 | Win/lose count 10.5/13.0 (-2.5)
Epoch 012/020 | Loss 0.2506 | Win/lose count 3.5/8.0 (-4.5)
Epoch 013/020 | Loss 0.2877 | Win/lose count 3.5/5.0 (-1.5)
Epoch 014/020 | Loss 0.3208 | Win/lose count 7.0/9.0 (-2.0)
Epoch 015/020 | Loss 0.1360 | Win/lose count 7.5/16.0 (-8.5)
Epoch 016/020 | Loss 0.1929

In [34]:
# We change the number of epochs for computional cost reasons
learning_rate = [0.01, 0.05, 0.2]
for l_r in learning_rate:
    print("\n For learning rate {0}".format(l_r)  )
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent = DQN_FC(lr=l_r, grid_size= size,  epsilon = 0.1, memory_size=2000, batch_size = 32)
    train(agent, env, 4, prefix='fc_train')


 For learning rate 0.01
Epoch 000/004 | Loss 0.0298 | Win/lose count 5.5/13.0 (-7.5)
Epoch 001/004 | Loss 0.0198 | Win/lose count 10.5/7.0 (3.5)
Epoch 002/004 | Loss 0.0659 | Win/lose count 4.5/8.0 (-3.5)
Epoch 003/004 | Loss 0.0180 | Win/lose count 4.0/13.0 (-9.0)
Final score: -4.125

 For learning rate 0.05
Epoch 000/004 | Loss 40.2525 | Win/lose count 10.5/16.0 (-5.5)
Epoch 001/004 | Loss 0.0098 | Win/lose count 6.5/6.0 (0.5)
Epoch 002/004 | Loss 0.0588 | Win/lose count 5.5/6.0 (-0.5)
Epoch 003/004 | Loss 0.0522 | Win/lose count 8.0/13.0 (-5.0)
Final score: -2.625

 For learning rate 0.2
Epoch 000/004 | Loss 152880.7031 | Win/lose count 3.0/1.0 (2.0)
Epoch 001/004 | Loss 1824.1357 | Win/lose count 4.0/4.0 (0.0)
Epoch 002/004 | Loss 1591.5972 | Win/lose count 1.0/4.0 (-3.0)
Epoch 003/004 | Loss 224.2659 | Win/lose count 2.5/1.0 (1.5)
Final score: 0.125


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [44]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        filters = 70
        kernel_size = (2,2)
        
        model = Sequential()
        model.add(Conv2D(filters, kernel_size, input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(filters, kernel_size, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        #print(model.summary())
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [45]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1,grid_size = size, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss nan | Win/lose count 4.0/1.0 (3.0)
Epoch 001/020 | Loss nan | Win/lose count 1.0/2.0 (-1.0)
Epoch 002/020 | Loss nan | Win/lose count 2.5/2.0 (0.5)
Epoch 003/020 | Loss nan | Win/lose count 0.5/3.0 (-2.5)
Epoch 004/020 | Loss nan | Win/lose count 0.5/2.0 (-1.5)
Epoch 005/020 | Loss nan | Win/lose count 3.0/0 (3.0)
Epoch 006/020 | Loss nan | Win/lose count 3.5/8.0 (-4.5)
Epoch 007/020 | Loss nan | Win/lose count 2.5/2.0 (0.5)
Epoch 008/020 | Loss nan | Win/lose count 3.5/7.0 (-3.5)
Epoch 009/020 | Loss nan | Win/lose count 3.0/3.0 (0.0)
Epoch 010/020 | Loss nan | Win/lose count 1.0/0 (1.0)
Epoch 011/020 | Loss nan | Win/lose count 1.0/5.0 (-4.0)
Epoch 012/020 | Loss nan | Win/lose count 6.5/4.0 (2.5)
Epoch 013/020 | Loss nan | Win/lose count 2.0/1.0 (1.0)
Epoch 014/020 | Loss nan | Win/lose count 1.0/0 (1.0)
Epoch 015/020 | Loss nan | Win/lose count 2.0/2.0 (0.0)
Epoch 016/020 | Loss nan | Win/lose count 1.0/5.0 (-4.0)
Epoch 017/020 | Loss nan | Win/lose count 4.5/4

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [46]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(lr=.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(lr=.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('\nTest of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\nTest of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')


Test of the CNN
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.0/3.0. Average score (-1.0)
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 3.0/0. Average score (0.0)
Win/lose count 0/1.0. Average score (-0.2)
Win/lose count 1.0/0. Average score (0.0)
Win/lose count 0/2.0. Average score (-0.2857142857142857)
Win/lose count 2.0/1.0. Average score (-0.125)
Win/lose count 2.5/0. Average score (0.16666666666666666)
Win/lose count 0/0. Average score (0.15)
Win/lose count 1.0/1.0. Average score (0.13636363636363635)
Win/lose count 1.0/2.0. Average score (0.041666666666666664)
Win/lose count 2.0/0. Average score (0.19230769230769232)
Win/lose count 0.5/0. Average score (0.21428571428571427)
Win/lose count 0/0. Average score (0.2)
Win/lose count 2.5/2.0. Average score (0.21875)
Win/lose count 1.0/2.0. Average score (0.14705882352941177)
Win/lose count 0.5/0. Average score (0.16666666666666666)
Win/lose count 1.0/2.0. Average score (0.10526315789473684)
Win/lose count 0/0. A

In [47]:
HTML(display_videos('cnn_test10.mp4'))

In [48]:
HTML(display_videos('fc_test10.mp4'))

The first thing we can observe is that the result is really better with the CNN than with the standard NN. However, the exploration rate is very low, which means we can improve our performance if our agent does more exploration. 

When we change the temperature, the final score is higher, but it is biased because by doing so we increase the probability of having cheese and we lower the probility of poisonious cells

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [49]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # Add an epsilon
            agent.set_epsilon(np.float(agent.get_epsilon())*0.995)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4       
        env.draw(prefix+str(e))
        
        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))

        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        self.board = np.zeros([grid_size,grid_size])
        self.position = np.zeros([grid_size,grid_size])
        
        # Add a malus position
        self.malus_position = np.zeros([grid_size,grid_size])
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def get_t(self):
        return self.t
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
    ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]

        self.malus_position[self.x, self.y] = 0.01
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))
        
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.malus_position[self.x, self.y] = 0

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state
        

In [50]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss nan | Win/lose count 2.5/6.799999999999963 (-4.299999999999963)
Epoch 001/020 | Loss nan | Win/lose count 1.0/6.87999999999996 (-5.87999999999996)
Epoch 002/020 | Loss nan | Win/lose count 1.49/1.9400000000000015 (-0.4500000000000015)
Epoch 003/020 | Loss nan | Win/lose count 0.5/1.9600000000000015 (-1.4600000000000015)
Epoch 004/020 | Loss nan | Win/lose count 0.5/4.949999999999958 (-4.449999999999958)
Epoch 005/020 | Loss nan | Win/lose count 0/2.010000000000001 (-2.010000000000001)
Epoch 006/020 | Loss nan | Win/lose count 2.0/2.8999999999999817 (-0.8999999999999817)
Epoch 007/020 | Loss nan | Win/lose count 0.98/2.98999999999998 (-2.00999999999998)
Epoch 008/020 | Loss nan | Win/lose count 0/2.9999999999999796 (-2.9999999999999796)
Epoch 009/020 | Loss nan | Win/lose count 0.5/4.979999999999958 (-4.479999999999958)
Epoch 010/020 | Loss nan | Win/lose count 1.5/3.919999999999959 (-2.419999999999959)
Epoch 011/020 | Loss nan | Win/lose count 1.0/2.939999999999981

In [51]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 0.49/1.9900000000000015. Average score (-1.5000000000000016)
Win/lose count 0.98/2.98999999999998. Average score (-1.7549999999999906)
Win/lose count 1.96/3.969999999999959. Average score (-1.83999999999998)
Win/lose count 1.47/2.96999999999998. Average score (-1.7549999999999801)
Win/lose count 0/3.0099999999999794. Average score (-2.00599999999998)
Win/lose count 0.49/2.98999999999998. Average score (-2.0883333333333134)
Win/lose count 0.49/4.969999999999959. Average score (-2.429999999999977)
Win/lose count 0/2.010000000000001. Average score (-2.37749999999998)
Win/lose count 2.45/2.9599999999999804. Average score (-2.16999999999998)
Win/lose count 1.47/2.97999999999998. Average score (-2.10399999999998)
Win/lose count 0/2.0000000000000013. Average score (-2.0945454545454365)
Win/lose count 0/2.9599999999999804. Average score (-2.1666666666666483)
Win/lose count 1.97/2.96999999999998. Average score (-2.0769230769230584)
Win/lose count 0.49/2.0000000000000013. Average 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***